<a href="https://colab.research.google.com/github/hewenpang/finetune_path/blob/main/Bert_Finetune_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import transformers
from transformers import AutoTokenizer

In [ ]:

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
tokenizer

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [ ]:
'''
autotokenizer and automodel 推理
'''
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
import torch
from datasets import load_dataset
!pip install evaluate
import evaluate
from torch.utils.data import DataLoader
from tqdm import tqdm
model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-uncased')
model.eval()
model.to('cuda')
test_dataset = dataset['test']

correct_count = 0
total_count = 0

for sample in tqdm(test_dataset):
    text = sample["text"]
    label = sample["label"]

    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)

    input_ids = encoded_input['input_ids'].to('cuda')
    attention_mask = encoded_input['attention_mask'].to('cuda')


    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        if predicted_class == label:
          correct_count += 1
        total_count += 1

accuracy = correct_count / total_count
print(f"在测试集上的准确率为: {accuracy:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 50000/50000 [12:00<00:00, 69.35it/s]

在测试集上的准确率为: 0.2000


In [ ]:
# for batch in test_dataloader:
#   for key,value in batch.items():
      # print(key)
      # print(value)
  # break

label
text
input_ids
token_type_ids
attention_mask


In [ ]:
'''
berttokenizer and BertForSequenceClassification 推理
'''
from transformers import BertForSequenceClassification,BertTokenizer,AutoModelForSequenceClassification,BertModel
import torch
!pip install datasets
from datasets import load_dataset

from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

dataset = load_dataset("yelp_review_full")
test_dataset = dataset['test']
# model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

bert_model.eval()
device = torch.device('cuda')
bert_model.to(device)
test_dataset = dataset['test']

correct_count = 0
total_count = 0

for sample in tqdm(test_dataset):
    text = sample["text"]
    label = sample["label"]

    encoded_input = bert_tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    # encoded_input = {key: value.to(device) for key, value in encoded_input.items()}
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    with torch.no_grad():
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        if predicted_class == label:
          correct_count += 1
        total_count += 1

accuracy = correct_count / total_count
print(f"在测试集上的准确率为: {accuracy:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 50000/50000 [09:46<00:00, 85.31it/s]

在测试集上的准确率为: 0.2000


In [1]:
from google.colab import drive
drive.mount('/content/ryan_drive')

Mounted at /content/ryan_drive


In [2]:
'''
使用trainer finetune 训练bert模型
'''
from transformers import AutoModelForSequenceClassification,AutoTokenizer
import torch
!pip install datasets
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import TrainingArguments
import numpy as np
!pip install evaluate
import evaluate
from transformers import Trainer
model = AutoModelForSequenceClassification.from_pretrained('google-bert/bert-base-uncased',num_labels=5, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

dataset = load_dataset("yelp_review_full")
def tokenize_function(examples):
    return tokenizer(examples["text"],padding='max_length', truncation=True)

encoded_dataset = dataset.map(tokenize_function,batched=True)

small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = encoded_dataset["test"].shuffle(seed=42).select(range(5000))

metric = evaluate.load("accuracy")

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fss

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch",run_name='finetune_bert')
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model(output_dir='/content/ryan_drive/MyDrive/Models')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.127300,1.044014,0.535600
2,0.813900,0.992442,0.591800
3,0.566200,1.280402,0.592000


In [4]:
'''微调模型推理'''
model = AutoModelForSequenceClassification.from_pretrained('/content/ryan_drive/MyDrive/Models')
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
model.eval()
device = torch.device('cuda')
model.to(device)
test_dataset = dataset['test']

correct_count = 0
total_count = 0

for sample in tqdm(test_dataset):
    text = sample["text"]
    label = sample["label"]

    encoded_input = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True)
    # encoded_input = {key: value.to(device) for key, value in encoded_input.items()}
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        if predicted_class == label:
          correct_count += 1
        total_count += 1

accuracy = correct_count / total_count
print(f"在测试集上的准确率为: {accuracy:.4f}")

100%|██████████| 50000/50000 [12:41<00:00, 65.70it/s]

在测试集上的准确率为: 0.5978
